# Sentiment analysis

## TF-IDF measure

Cтатистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.

Мера TF-IDF является произведением двух сомножителей: $$ tf\text{-}idf(t,d,D)= tf(t,d) \times idf(t,D)  $$.
Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах.

### TF

TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Оценивается важность слова $t_i$ в пределах отдельного документа.

$ \Large tf(t,d) = \frac{n_t}{\sum_k n_k} $,

где $n_t$ - есть число вхождений слова $t$ в документ, а в знаменателе — общее число слов в данном документе.

### IDF
IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции. Учёт IDF уменьшает вес широкоупотребительных слов.

 $ \Large idf(t, D) = log \frac{|D|} { | \{ d_i \in D \mid t \in d_i \} | } $,
 
где
* $|D|$ — число документов в коллекции;
* $| \{ d_i \in D \mid t \in d_i \} |$ — число документов из коллекции $ D $, в которых встречается $t$ (когда $n_t \neq 0 $).

## Dataset

In [1]:
from nltk.tokenize import word_tokenize
import eli5
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()


In [2]:
df = pd.read_csv('women-clothing-accessories.3-class.balanced.csv', encoding='utf8', sep='\t')
df


,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [3]:
df['sentiment'].value_counts()


sentiment
negative    30000
neautral    30000
positive    30000
Name: count, dtype: int64

In [4]:
#оставим только 2 класса: позитивные и негативные
df = df[df['sentiment'] != 'neautral']


In [5]:
df.iloc[0]['review']


'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

## Prepocessing

In [6]:
# удаление знаков препинания
df['review_processed'] = df['review'].apply (lambda x: re.sub(r'[^\w\s]', '', x)).values
df


C:\Users\Ilya Solopov\AppData\Local\Temp\ipykernel_2320\1979955618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_processed'] = df['review'].apply (lambda x: re.sub(r'[^\w\s]', '', x)).values


,review,sentiment,review_processed
0,качество плохое пошив ужасный (горловина напер...,negative,качество плохое пошив ужасный горловина напере...
1,"Товар отдали другому человеку, я не получила п...",negative,Товар отдали другому человеку я не получила по...
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative,Ужасная синтетика Тонкая ничего общего с предс...
3,"товар не пришел, продавец продлил защиту без м...",negative,товар не пришел продавец продлил защиту без мо...
4,"Кофточка голая синтетика, носить не возможно.",negative,Кофточка голая синтетика носить не возможно
...,...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive,сделано достаточно хорошо на ткани сделан рису...
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive,Накидка шикарная Спасибо большое провдо линяет...
89997,спасибо большое ) продовца рекомендую.. заказа...,positive,спасибо большое продовца рекомендую заказала ...
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive,Очень довольна заказом Меньше месяца в РБ Кур...


In [7]:
# приводим к нижнему регистру
df['review_processed'] = df['review_processed'].apply(lambda x: x.lower()).values


C:\Users\Ilya Solopov\AppData\Local\Temp\ipykernel_2320\239575385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_processed'] = df['review_processed'].apply(lambda x: x.lower()).values


In [8]:
# токенизация
df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))


100%|██████████| 60000/60000 [00:18<00:00, 3161.93it/s]
C:\Users\Ilya Solopov\AppData\Local\Temp\ipykernel_2320\277047207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_processed'] = df['review_processed'].progress_apply(lambda x: word_tokenize(x))


In [9]:
df['review_processed'].iloc[1]


['товар',
 'отдали',
 'другому',
 'человеку',
 'я',
 'не',
 'получила',
 'посылку',
 'ладно',
 'хоть',
 'деньги',
 'вернули']

## Lemmatization

In [10]:
morph = pymorphy2.MorphAnalyzer()


In [11]:
df['review_lemmatized'] = df['review_processed'].progress_apply(lambda text: [morph.parse(word)[0].normal_form for word in text]).values


100%|██████████| 60000/60000 [03:23<00:00, 294.33it/s]
C:\Users\Ilya Solopov\AppData\Local\Temp\ipykernel_2320\3007154353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review_lemmatized'] = df['review_processed'].progress_apply(lambda text: [morph.parse(word)[0].normal_form for word in text]).values


## Feature extraction using TF-IDF

In [23]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2)) #коллокации


In [24]:
X = vectorizer.fit_transform(df['review_lemmatized'].apply(lambda x: ' '.join(x)))


In [25]:
# (samples, features)
X.shape


(60000, 396100)

## Training

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=26)


In [64]:
model = LogisticRegression()


In [65]:
model.fit(X_train, y_train)


d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


LogisticRegression()

## Testing

In [66]:
predicts = model.predict_proba(X_test)[:, 1]
metrics = roc_auc_score(y_test, predicts)


d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [67]:
print(f"ROC AUC score: {metrics:.3%}")

ROC AUC score: 97.661%


In [68]:
# визулизация весов
eli5.show_weights(estimator=model, feature_names=list(vectorizer.get_feature_names_out()), top=(20,20))

Weight?,Feature
+10.731,отличный
+10.107,хороший
+8.287,супер
+7.749,спасибо
+7.487,немного
+7.130,хорошо
+6.800,классный
+6.799,отлично
+6.477,приятный
+6.399,довольный


## Gridsearch

In [69]:
parameters = {'C' : [0.25, 0.5, 0.75, 1, 5, 10], 'max_iter' : [50, 100, 150]}
clf = GridSearchCV(model, parameters)

In [70]:
clf.fit(X_train, y_train)

d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not 

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.25, 0.5, 0.75, 1, 5, 10],
                         'max_iter': [50, 100, 150]})

In [71]:
gs_C, gs_max_iter = clf.best_params_.values()
gs_best_model = LogisticRegression(C=gs_C,max_iter=gs_max_iter)

In [72]:
gs_best_model.fit(X_train, y_train)

d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(C=10, max_iter=150)

In [73]:
predicts = gs_best_model.predict_proba(X_test)[:, 1]
metrics = roc_auc_score(y_test, predicts)
print(f"ROC AUC score: {metrics:.3%}")

ROC AUC score: 98.044%


d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


## Randomsearch

In [80]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

distributions = {'C' : uniform(0.25,10), 'max_iter' : [50, 100, 150]}
clf = RandomizedSearchCV(model, distributions)

In [81]:
clf.fit(X_train,y_train)

d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not 

RandomizedSearchCV(estimator=LogisticRegression(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001A8D97C3AF0>,
                                        'max_iter': [50, 100, 150]})

In [82]:
rs_C, rs_max_iter = clf.best_params_.values()
rs_best_model = LogisticRegression(C=rs_C, max_iter=rs_max_iter)

In [83]:
rs_best_model.fit(X_train, y_train)

d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression(C=10.051086952243828, max_iter=150)

In [84]:
predicts = rs_best_model.predict_proba(X_test)[:, 1]
metrics = roc_auc_score(y_test, predicts)
print(f"ROC AUC score: {metrics:.3%}")

ROC AUC score: 98.045%


d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
d:\Program Files (x86)\Python\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


При использовании Randomsearch метрика та же (даже чуть выше), однако скорость сходимости больше примерно в 1.3 раза.